# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('dataset/cifar/train/*.png')[:100]
test_paths = glob('dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Training

http://localhost:6006

In [14]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

Epoch 1/5
3/3 [==============================] - 1s 443ms/step - loss: 2.3912 - accuracy: 0.0588 - val_loss: 2.3225 - val_accuracy: 0.0625
Epoch 2/5
3/3 [==============================] - 0s 63ms/step - loss: 2.3247 - accuracy: 0.1250 - val_loss: 2.2966 - val_accuracy: 0.1146
Epoch 3/5
3/3 [==============================] - 0s 51ms/step - loss: 2.2447 - accuracy: 0.1912 - val_loss: 2.3112 - val_accuracy: 0.1146
Epoch 4/5
3/3 [==============================] - 0s 48ms/step - loss: 2.2804 - accuracy: 0.1029 - val_loss: 2.3198 - val_accuracy: 0.0729
Epoch 5/5
3/3 [==============================] - 0s 53ms/step - loss: 2.2029 - accuracy: 0.2353 - val_loss: 2.3349 - val_accuracy: 0.0729


# Saving Model

In [15]:
save_path = 'my_model.h5'

In [16]:
# model.save(save_path)
model.save(save_path, include_optimizer=True)

In [17]:
model = tf.keras.models.load_model('my_model.h5')

# Saving Model - 2

In [18]:
model.save_weights('model_weights.h5')

In [19]:
with open('model_arch.json', 'w') as f:
    f.write(model.to_json())

In [20]:
from tensorflow.keras.models import model_from_json

In [21]:
with open('model_arch.json', 'r') as f:
    model = model_from_json(f.read())

In [22]:
model.load_weights('model_weights.h5')

# model.h5 들여다보기

In [23]:
import h5py

In [24]:
model_file = h5py.File('my_model.h5', 'r+')

In [25]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [26]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [27]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [28]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [29]:
weight = np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])
weight

array([[[[ 1.21366665e-01,  2.99348496e-04,  7.99982399e-02,
           1.31558940e-01,  9.98735726e-02, -3.78046669e-02,
           4.11765613e-02, -2.44274884e-02,  1.02012828e-01,
           1.09860599e-01, -2.19347328e-02, -1.22738324e-01,
          -2.08761860e-02, -2.06623469e-02,  7.77631858e-03,
          -9.51249078e-02,  1.30009338e-01, -7.98405265e-04,
           5.44179976e-03, -4.14365716e-02,  1.30681306e-01,
          -6.81053996e-02, -1.04894601e-01, -1.32618874e-01,
          -1.33968383e-01, -1.05354279e-01,  4.43992242e-02,
           2.54455600e-02, -3.46261598e-02, -2.07972080e-02,
           7.52578527e-02,  1.03738740e-01],
         [-2.40270086e-02, -3.96802984e-02,  1.24814920e-01,
           1.25878394e-01,  9.95108038e-02,  3.24664414e-02,
          -5.00251986e-02, -7.99277350e-02,  6.92798421e-02,
           6.47290498e-02,  5.30928858e-02,  9.59885418e-02,
          -2.08241194e-02, -5.33282682e-02,  8.89126286e-02,
          -5.39698973e-02, -1.01592094e-